In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -l ~/.kaggle/kaggle.json

-rw------- 1 root root 65 Jan 17 18:20 /root/.kaggle/kaggle.json


In [2]:
!kaggle datasets download -d hrwarrior/iinduction-task-2025

Dataset URL: https://www.kaggle.com/datasets/hrwarrior/iinduction-task-2025
License(s): CC0-1.0
 97% 139M/144M [00:02<00:00, 31.0MB/s]
100% 144M/144M [00:02<00:00, 54.9MB/s]


In [3]:
!kaggle competitions download -c induction-task-2025

 83% 67.0M/80.9M [00:00<00:00, 126MB/s]
100% 80.9M/80.9M [00:00<00:00, 137MB/s]


In [4]:
import zipfile
zip_f = zipfile.ZipFile('/content/iinduction-task-2025.zip', 'r')
zip_f.extractall('/content')
zip_f.close()

In [5]:
import zipfile
zip_f = zipfile.ZipFile('/content/induction-task-2025.zip', 'r')
zip_f.extractall('/content')
zip_f.close()

In [6]:
!pip install pillow_heif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.5 MB/s eta 0:00:00


In [7]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D

In [9]:
import os
import cv2
from PIL import Image, UnidentifiedImageError
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/New_Data"
new_size = (256,256)
batch_size = 25

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.20)

def image_loader(img_path):
    try:
        with open(img_path, "rb") as f:
            img = Image.open(f).convert('RGB').resize(new_size)
            return img
    except UnidentifiedImageError:
        print(f"Error loading image: {img_path}")
        return None
    except Exception as e:
        print(f"Unexpected error loading image: {img_path}, Error: {e}")
        return None

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=new_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
)

validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=new_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
)

Found 474 images belonging to 2 classes.
Found 118 images belonging to 2 classes.


In [10]:
image_folder = "/content/New_Data/Real"
output_folder = "/content/Real_resized"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, new_size)
        cv2.imwrite(os.path.join(output_folder, filename), img_resized)

In [11]:
image_folder = "/content/New_Data/AI"
output_folder = "/content/AI_resized"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error loading image: {img_path}")
            continue
        img_resized = cv2.resize(img, new_size)
        cv2.imwrite(os.path.join(output_folder, filename), img_resized)

Error loading image: /content/New_Data/AI/image_326.jpg
Error loading image: /content/New_Data/AI/image_530.jpg


In [12]:
import numpy as np

image_folder = "/content/Real_resized"

image_array_list = []

for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(image_folder, filename)
        img = Image.open(img_path).resize((256, 256))
        img_array = np.array(img)
        image_array_list.append(img_array)

real_array = np.stack(image_array_list)
print(f"Final Shape of Array: {real_array.shape}")

Final Shape of Array: (296, 256, 256, 3)


In [13]:

image_folder = "/content/AI_resized"

image_array_list = []

for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(image_folder, filename)
        img_array = np.array(img)
        image_array_list.append(img_array)

ai_array = np.array(image_array_list)
print(f"Final Shape of Array: {ai_array.shape}")


Final Shape of Array: (294, 256, 256, 3)


In [14]:
label_ai = np.zeros(len(ai_array))
label_real = np.ones(len(real_array))

X = np.concatenate((ai_array, real_array), axis=0)
Y = np.concatenate((label_ai, label_real), axis=0)

print(f"Shape of X: {X.shape}")
print(f"Shape of Y: {Y.shape}")

Shape of X: (590, 256, 256, 3)
Shape of Y: (590,)


In [16]:

model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(512, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.4),

    GlobalAveragePooling2D(),

    Dense(128, activation='relu'),
    Dropout(0.4),

    Dense(1, activation='sigmoid')
])

In [17]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=11,
    batch_size=27
)

Epoch 1/11


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 5/19 ━━━━━━━━━━━━━━━━━━━━ 5s 393ms/step - accuracy: 0.4675 - loss: 1.3944

In [ ]:
model.evaluate(validation_generator)

5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 614ms/step - accuracy: 0.4681 - loss: 0.9813


[0.9422107338905334, 0.504273533821106]

In [ ]:
image_folder = "/content/Test_Images"
output_folder = "/content/Test_resized"
os.makedirs(output_folder, exist_ok=True)
new_size=(256,256)
a=0
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)

        img_resized = cv2.resize(img, new_size)
        cv2.imwrite(os.path.join(output_folder, filename), img_resized)
        a+=1

print("All images have been reshaped and saved.")
print(a)

All images have been reshaped and saved.
200


In [ ]:
image_folder = "/content/Test_resized"

def natural_sort_key(filename):
    import re
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

image_filenames = sorted(
    [f for f in os.listdir(image_folder) if f.endswith((".jpg", ".png", ".jpeg"))],
    key=natural_sort_key
)

image_array_list = []
image_size=(256,256)
for filename in image_filenames:
    img_path = os.path.join(image_folder, filename)
    img = Image.open(img_path).resize(image_size)
    img_array = np.array(img)
    image_array_list.append(img_array)

test_array = np.stack(image_array_list)

print(f"Final array shape: {test_array.shape}")


Final array shape: (200, 256, 256, 3)


In [ ]:
test_array=test_array/255.0
prediction = model.predict(test_array)

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 201ms/step


In [ ]:

import csv

image_ids = [f'image_{i}' for i in range(200)]

labels = ['Real' if pred > 0.5 else 'AI' for pred in prediction]

if len(image_ids) != len(labels):
    print("Error: Mismatch between number of image IDs and predictions.")
else:
    with open('Final.csv', mode='w', newline='') as file:
        writer = csv.writer(file)

        writer.writerow(['Id', 'Label'])

        for img_id, label in zip(image_ids, labels):
            writer.writerow([img_id, label])

    print("Submission file created: Final.csv")

Submission file created: Final.csv


In [ ]:

!kaggle competitions submit -c induction-task-2025 -f Final.csv -m "Message"

100% 3.03k/3.03k [00:00<00:00, 4.81kB/s]
Successfully submitted to Gradient Hunt 2.0: The Saga 